In [1]:
import pandas as pd
import os, sys, pickle
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.utils import multi_gpu_model
import tensorflow as tf
import subprocess, argparse

sys.path.append('/lustre/eaglefs/projects/pvdetect/SkyImage/SolarNet4/Git/Code')
from get_model import *
from get_generators import *


# def get_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--img_size", type=int, default = 256, help="Input sky image resolution")
#     parser.add_argument("--no_img", type=int, default= 2, help="Length of input image sequence")
#     parser.add_argument("--train_batchsize", type=int, default= 64)
#     parser.add_argument("--validation_batchsize", type=int, default= 64)
#     parser.add_argument("--test_batchsize", type=int, default= 1)
#     args = parser.parse_args()
#     return args
# args = get_args()

# get parameters that should be received from the terminal command line for HPC job submissions
img_size = 128
no_img = 2
train_batchsize = 8
validation_batchsize = 8
test_batchsize = 1



Using TensorFlow backend.


In [2]:
file = open(os.path.join('/lustre/eaglefs/projects/pvdetect/SkyImage/SolarNet4/Git/Data', 'Data_3Days_eagle.pkl'),'rb')
df_train = pickle.load(file)
df_validation = pickle.load(file)
df_test = pickle.load(file)
file.close()

# check the data frame shape
print(df_train.shape)
print(df_validation.shape)
print(df_test.shape)

# look into the data frame
df_train.head


(54, 2)
(54, 2)
(54, 2)


<bound method NDFrame.head of                                                 Image     Target
0   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   1.743735
1   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   1.491311
2   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   1.263644
3   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   1.041370
4   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   0.831888
5   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   0.647980
6   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   0.872622
7   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   0.861904
8   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   0.791358
9   [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   0.970264
10  [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   0.902137
11  [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   0.890267
12  [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   1.028943
13  [/lustre/eaglefs/projects/pvdetect/SkyImage/Da...   1.02

In [3]:
params1 = {'batch_size': train_batchsize,
           'dim': (img_size, img_size, 3 * no_img),
           'channel_IMG': 3,
           'shuffle': True,
           'iftest': False}
params2 = {'batch_size': validation_batchsize,
           'dim': (img_size, img_size, 3 * no_img),
           'channel_IMG': 3,
           'iftest': False}
params3 = {'batch_size': test_batchsize,
           'dim': (img_size, img_size, 3 * no_img),
           'channel_IMG': 3,
           'shuffle': False,
           'iftest': False}
# build data generators
train_generator = DataGenerator_SCNN(df_train, **params1)
validation_generator = DataGenerator_SCNN(df_validation, **params2)
test_generator = DataGenerator_SCNN(df_test, **params3)

In [4]:
# build the SolarNet
conv_base = SCNN(input_shape=(img_size, img_size, 3 * no_img))
conv_base.summary() # check base model, a SCNN is used here for demonstration

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation='linear'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1, activation='linear'))
model.summary()

# compile model
optimizer = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
class MyModelCheckPoint(ModelCheckpoint):
    def __init__(self, singlemodel, *args, **kwargs):
        self.singlemodel = singlemodel
        super(MyModelCheckPoint, self).__init__(*args, **kwargs)
    def on_epoch_end(self, epoch, logs=None):
        self.model = self.singlemodel
        super(MyModelCheckPoint, self).on_epoch_end(epoch, logs)
checkpointer1 = MyModelCheckPoint(model, filepath="BestSingleModel.hdf5", save_best_only=True, verbose=1)
# HPC parallel computing
# parallel_model = multi_gpu_model(model, gpus=n_GPUs)
# parallel_model.compile(loss='mean_absolute_error',  # mean_squared_error
#                        optimizer=optimizer,
#                        metrics=['mae'])
model.compile(loss='mean_absolute_error',  # mean_squared_error
              optimizer=optimizer,
              metrics=['mae'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 64)      3520      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 128)       0         
______________________________________

In [5]:
# Train the SolarNet
os.chdir('/lustre/eaglefs/projects/pvdetect/SkyImage/SolarNet4/Git/Results')
history = model.fit_generator(train_generator,
                              steps_per_epoch=int(df_train.shape[0] / train_batchsize),
                              epochs=30,
                              validation_data=validation_generator,
                              validation_steps=int(df_validation.shape[0] / validation_batchsize),
                              callbacks=[checkpointer1],
                              verbose=1,
                              workers=6,
                              use_multiprocessing=False)
model.save('Last_SingleModel.hdf5')

# predict
try:
    model.load_weights('BestSingleModel.hdf5')
    print('Succss in loading the best single model')
except:
    print('Fail to load the best single model')
    pass

y_hat = model.predict_generator(generator=test_generator, steps=df_test.shape[0] / test_batchsize)



2021-11-10 15:49:49.430750: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-10 15:49:49.430797: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-10 15:49:49.430839: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (europa-int): /proc/driver/nvidia/version does not exist
2021-11-10 15:49:49.431448: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-11-10 15:49:49.449293: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2021-11-10 15:49:49.458561: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ef691a1280 initialized for platform Host (this does not guarant


Epoch 1/30
6/6 [==============================] - 6s 999ms/step - loss: 1.2741 - mae: 1.2741 - val_loss: 1.1037 - val_mae: 1.3242

Epoch 00001: val_loss improved from inf to 1.10365, saving model to BestSingleModel.hdf5
Epoch 2/30
6/6 [==============================] - 5s 893ms/step - loss: 1.0204 - mae: 1.0204 - val_loss: 0.9491 - val_mae: 1.2183

Epoch 00002: val_loss improved from 1.10365 to 0.94906, saving model to BestSingleModel.hdf5
Epoch 3/30
6/6 [==============================] - 5s 893ms/step - loss: 1.0423 - mae: 1.0423 - val_loss: 0.8143 - val_mae: 0.8932

Epoch 00003: val_loss improved from 0.94906 to 0.81433, saving model to BestSingleModel.hdf5
Epoch 4/30
6/6 [==============================] - 5s 899ms/step - loss: 0.5750 - mae: 0.5750 - val_loss: 0.1677 - val_mae: 0.3913

Epoch 00004: val_loss improved from 0.81433 to 0.16773, saving model to BestSingleModel.hdf5
Epoch 5/30
6/6 [==============================] - 5s 900ms/step - loss: 0.2992 - mae: 0.2992 - val_loss: 0.